In [15]:
import pandas as pd
import glob
import gc
import numpy as np
import pyarrow as pa

## Functions and Variables

In [1]:
def find_files(path, pattern):
    files = glob.glob(f'{path}/{pattern}')
    # print(files)
    # print(len(files))

    return files

In [2]:
# Creates a column called index in the dataframe
# this column will have the format:
# prefix + 0000001 (Ex. 310.000.001) to prefix + length of dataframe (Ex. 312.123.942)

def add_index(df, prefix):
    size = len(df)
    prefix = prefix * 10**len(str(size)) + 1

    df['index'] = range(prefix, prefix + size)
    df.set_index('index', inplace=True)

In [3]:
def safe_delete(df, list_columns):
    for item in list_columns:
        if item in df.columns:
            df = df.drop(item, axis=1)
    
    return df

In [7]:
files = find_files('D:\\_repositories\\Aggregation\\SIM\\1. Concatenado', 
                   '*.csv')
files

['D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2012.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2013.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2014.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2015.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2016.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2017.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2018.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2019.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2020.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2021.csv',
 'D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2022.csv']

---

## Processing

In [26]:
inicial_ano = 12 # ano em processamento
lista_incompleto = []

for item in files:
    df = pd.read_csv(item)

    # Index
    add_index(df, int('1' + str(inicial_ano)))

    # Deleting past index columns
    df = safe_delete(df, ['Unnamed: 0', 'Unnamed: 0.1'])

    # Dtype
    df = df.convert_dtypes()
    treat_columns(df)
    df = df.drop_duplicates()

    # Saving to parquet
    try: 
        df.to_parquet(f'D:/_repositories/Aggregation/SIM/2. Index e Parquet/SIM20{inicial_ano}.parquet')
    except pa.ArrowInvalid:
        lista_incompleto.append(item)
    
    inicial_ano += 1

    # Memory safety
    del df
    gc.collect()

C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\321044736.py:5: DtypeWarning: Columns (24,52,53,54,63,64,65,66,67,91,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\321044736.py:5: DtypeWarning: Columns (24,91,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\321044736.py:5: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\321044736.py:5: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\321044736.py:5: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)
C:\Users\

In [19]:
lista_incompleto

['D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2014.csv']

---

## Individual file handling

#### SIM2014.csv

In [28]:
df = pd.read_csv('D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2014.csv')
add_index(df, int('1' + str(14)))
df = safe_delete(df, ['Unnamed: 0', 'Unnamed: 0.1'])

C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\955476419.py:1: DtypeWarning: Columns (24,52,53,54,63,64,65,66,67,91,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\_repositories\\Aggregation\\SIM\\1. Concatenado\\SIM2014.csv')


In [30]:
df = df.convert_dtypes()
treat_columns(df)
df = df.drop_duplicates()

In [48]:
df.to_parquet('D:/_repositories/Aggregation/SIM/2. Index e Parquet/SIM2014.parquet')

##### Coluna CRM

In [37]:
df['CRM'].loc[df['CRM'] == 'CRM 59299'] = 59299
df['CRM'].loc[df['CRM'] == '9852- SVO'] = 9852

df['CRM'] = df['CRM'].astype('Float64')
df['CRM'] = df['CRM'].astype('Int64')

C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\1533685486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CRM'].loc[df['CRM'] == 'CRM 59299'] = 59299
C:\Users\ricmo\AppData\Local\Temp\ipykernel_5408\1533685486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CRM'].loc[df['CRM'] == '9852- SVO'] = 9852
